## The Trivia API and The Quizzler App
- [Trivia](https://opentdb.com/) API를 이용해서 지난시간 복습하기
- 해당 API를 통해서 Quizzler 앱 구축하기 

### 1. Trivia Question API
- Quizzler app을 만들기 위해 이전에 작업 했던 quiz quiz를 참조 
- 이번에는 Trivia API를 참조해서 데이터 불러오기 

In [ ]:
# quize_module/data.py
# Trivia Database API를 이용해서 데이터 불러오기 

# 모듈 불러오기 
import requests

# 파라미터 설정 
prameters = {
    "amount": 20,
    "type": "boolean"
}

response = requests.get("https://opentdb.com/api.php", params=prameters)   # 앤드포인트로 요청 받기 
response.raise_for_status()                                                # 응답코드 - 200이 아니면 예외를 발생 시킴 

question_data = response.json()["results"]                                  # Trivia API에서 가져온 데이터 저장

### 2. HTML 개체 언이스케이핑 
- Trivai API를 통해 데이터는 제대로 받았지만, 텍스트 안에 아래와 같은 형식의 문자가 같이 따라온다. 
    - "In **&** **quot;**Mario Kart 64**&** **quot;**, Waluigi is ..."
- 우리가 보고있는 것은 문자열이 아닌 HTML 개체이기 때문에 위와 같은 이상한 형식의 문자가 같이 따라온다. 
- [HTML Entities Docs](https://www.w3schools.com/html/html_entities.asp)를 보면 "몇 가지 유용한 HTML 문자 개체"란을 보면 위에 나타낸  **&** **quot;**가 '"'를 의미하는 것을 알수 있다.
- 그럼 이 HTML 개체를 어떻게 문자 결과로 나타낼까? 바로 html 모듈을 임포트하고, 해당 모듈에 있는 메소드인 "unescape"를 사용해서 우리가 받은 텍스트를 언이스케이핑 해야한다.

In [ ]:
# html 개체 언이스케이핑 방법 
import html 

html.unescape("unescape html entities text")

In [ ]:
# quize question text html 개체 언이스케이핑 하기 
# quiz_module/quiz_brain.py 

# QuizeBrain -> 퀴즈의 모든 질문과 퀴즈의 기능을 위한 Class 
# 기능 1: 사용자에게 다음 질문을 제시 
# 기능 2: 퀴즈의 정답을 맞췄는지 여부 확인 
# 기능 3: 퀴즈의 마지막 문제까지 도달했는지 여부 확인 

# 필요 속성: quetion_number - 퀴즈 문제 순서 / question_list - 질문 리스트  / score - 문제 점수
# 필요한 메소드 1: next_question() - 현재 퀴즈 질문 호출 및 입력할 정답 호출 기능
# 필요한 메소드 2: still_has_questions() - 뒤에 퀴즈가 남았는지 여부를 확인하는 기능
# 필요한 메소드 3: check_score() - 퀴즈 점수 체크를 위한 기능 
import html                                             # html 모듈 불러오기 

class QuizBrain:                                        # QuizBrain 이라는 클래스 생성 
    # 초기화 메소드 
    def __init__(self, q_list):                         # question_number, score(기본값이 0이기 때문에 여기서 생략), question_list, 두 속성을 초기화 하는 init 메소드 (입력 값 입력)
        self.question_number = 0                        # 속성 설정 1. 문제 번호를 위한 "question_number"
        self.score = 0                                  # 속성 설정 2. 점수를 메기기 위한 "score" 
        self.question_list = q_list                     # 속성 설정 3. quiz 데이터를 뽑기 위한 "question_list"
        self.current_question = None                    # 속성 설정 4. currenct_question = None 설정

    # still_has_questions() - 뒤에 퀴즈가 남았는지 여부를 확인하는 기능
    def still_has_questions(self):                               
        return self.question_number < len(self.question_list)    # 아래의 구조를 하나로 작성 
        # if self.question_number < len(self.question_list):                     # 리스트에 있는 질문의 개수만큼 반복 실행되길 원함 - question_number가 question_list의 길이 보다 적다면,
        #     return True                                                        # True를 반환 
        # else:                                                                  # question_number가 question_list의 길이 보다 크다면,
        #     return False                                                       # False를 반환 
    
    # next_question 메소드 - question_list에서 현재 질문 번호의 항목을 가져오고, input 함수를 이용해서 입력한 정답을 호출하는 기능
    def next_question(self):
        self.current_question = self.question_list[self.question_number]            # 현재 퀴즈 질문 - question_list[question_number]
        self.question_number += 1                                                   # 문제 번호 더하기 - 해당 설정을 하지 않으면, Q.0으로 넘버링이 됨 
        q_text = html.unescape(self.current_question.text)                          # question text 선언 후 html 언이스케이프
        user_answer = input(f"Q.{self.question_number}: {q_text} (True/False): ")   # 입력할 정답 호출 
        self.check_answer(user_answer)                                              # 입력할 정답의 값들을 check_answer로 전달 

    # check_answer 메소드 - 유저가 입력한 값과 정답을 비교하여 점수를 체크하는 기능 
    def check_answer(self, user_answer):
        correct_answer = self.current_question.answer          
        if user_answer.lower() == correct_answer.lower():          # 만약 유저가 입력한 정답과, 문제 정답이 같으면
            self.score += 1                                        # 점수 1씩 증가 
            print("You got it right!")                             # 결과 출력 
        else:                                                      # 만약 다르다면
            print("That's wrong.")                                 # 결과 출력 

        print(f"The correct answer was: {correct_answer}")                     # 문제의 정답 출력 
        print(f"Your current score is: {self.score}/{self.question_number}")   # 점수 출력 
        print("\n")                                                            # 다음 문제를 위해 줄 바꾸기 

### 3. 클래스 기반 Tkinter UI

In [ ]:
# quiz_module/ui.py

# UI 설정을 위한 클래스 

# 모듈 불러오기 
from tkinter import *

# 상수설정 - 테마 색상 
THEME_COLOR = "#375362"

# quiz UI를 위한 QuizInterface 클래스 선언 
class QuizInterface:
    
    # 초기값 설정
    def __init__(self):
        self.window = Tk()                                                        # tkinter 객체 선언 
        self.window.title("Quizzler")                                             # tkinter title 지정
        self.window.config(padx=20, pady=20, bg=THEME_COLOR)                      # pad(간격) 및 배경색 설정
        
        # Labels 
        self.score_label = Label(text="Score: 0", fg="white", bg=THEME_COLOR)     # Label 1 - 점수판 설정 
        self.score_label.grid(row=0, column=1)                                    # grid로 명시 
        
        # Canvas
        self.canvas = Canvas(width=300, height=250, bg="white")                   # canvas 객체 선언 - 선언시 크기 및 배경 색상 설정 
        self.question_text = self.canvas.create_text(                             # text판 선언 
                                                    150,                          # x값 
                                                    125,                          # y값 
                                                    text="Some Question Text",    # Text 
                                                    fill=THEME_COLOR,             # 배경색상 
                                                    font=("Arial", 20, "italic")  # 폰트 설정 
                                                    )
        self.canvas.grid(row=1, column=0, columnspan=2, pady=50)                  # grid로 명시 
        
        # buttons
        ture_image = PhotoImage(file="./img/true.png")                            # true 버튼에 사용할 이미지 불러오기 
        self.true_button = Button(image=ture_image, highlightthickness=0)         # 버튼 설정 - 이미지 설정 및 테두리 제거 
        self.true_button.grid(row=2, column=0)                                    # grid로 명시 
        
        false_image = PhotoImage(file="./img/false.png")                          # false 버튼에 사용할 이미지 불러오기 
        self.false_button = Button(image=false_image, highlightthickness=0)       # 버튼 설정 - 이미지 설정 및 테두리 제거 
        self.false_button.grid(row=2, column=1)                                   # gird로 명시 
        
        # 닫기 버튼을 누르기 전까지 계속 구동
        self.window.mainloop()

In [ ]:
# 18_quizzler_app.py(main.py)

from quiz_module.question_model import Question
from quiz_module.data import question_data
from quiz_module.quiz_brain import QuizBrain
from quiz_module.ui import QuizInterface

question_bank = []
for question in question_data:
    question_text = question["question"]
    question_answer = question["correct_answer"]
    new_question = Question(question_text, question_answer)
    question_bank.append(new_question)


quiz = QuizBrain(question_bank)
quiz_ui = QuizInterface()       # UI의 QuizInterface 클래스 불러오기

# while quiz.still_has_questions():
#     quiz.next_question()

print("You've completed the quiz")
print(f"Your final score was: {quiz.score}/{quiz.question_number}")

### 4. 파이썬 타이핑 - 타입 힌트와 화살표

- 예를 들어 우리가 age라는 이름 변수를 생성하려 한다면, "age=12"라고 한다.
- 하지만 데이터형을 선언하고 그대로 둘 수 있다. (아래의 예제처럼)

In [4]:
age: int
age = 12

12


- 나중에 어떤 시점이 되어서 사용자로부터 age를 받거나 우리가 그것을 얻으면, 그 시점에서 age 변수를 설정할 수 있다는 의미이다. 그리고 이 age는 위에 있는 데이터형과 일치해야한다.
- 추후 다른 데이터 타입으로 변수를 선언하게 되면, age가 정수타입이어야 한다 라고 알려준다.

In [ ]:
# 데이터 타입 설정 예시 1
age: int
name: str 
height: float 
is_human: bool 

In [5]:
# 데이터 타입 설정 예시 2 - 함수 선언시 
def police_check(age):
    if age > 18:
        car_drive = True
    else:
        car_drive = False 
    return car_drive

In [6]:
# 함수 출력 결과 
police_check(19)

True

In [7]:
# 데이터 타입 설정 예시 2 - 함수 선언시 매개변수 데이터 타입 설정 
def police_check(age:int):
    if age > 18:
        car_drive = True
    else:
        car_drive = False 
    return car_drive

In [10]:
# 함수 출력 결과 
police_check(19)

True

In [9]:
# 매개 변수를 잘못 지정했을 때
police_check("19")

TypeError: '>' not supported between instances of 'str' and 'int'

- 이렇게 하는 이유는 우리가 코드 실행시 문제 및 버그를 사전에 차단할 수 있기 때문이다.

- 추가적으로 함수 출력시 데이터형을 지정할 수 있다. 방법은 하이픈과 부등호로 화살표를 만들고, 출력하고자 하는 데이터 형을 지정하면 된다. 만약 return에 다른 데이터 타입을 반환하는 경우 힌트를 준다.
- 이것을 타입 힌트라고 부른다. 

In [11]:
# 데이터 타입 설정 예시 3 - 함수 출력시 데이터형 지정 -> bool으로 
def police_check(age:int) -> bool:
    if age > 18:
        car_drive = True
    else:
        car_drive = False 
    return car_drive

In [12]:
# 매개 변수를 잘못 지정했을 때
police_check(19)

True

In [17]:
# 데이터 타입 설정 예시 4 - 타입 힌트 예제 
def greeting(name: str) -> str:
    return "Hello " + name

In [18]:
greeting("Gilbert")

'Hello Gilbert'